In [10]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.models import load_model

import mlflow
from mlflow import pyfunc
import mlflow.tensorflow

In [11]:
train_dir = r'F:\Datasets\dog-breed-identification\train'

In [19]:
def run_training(model=None):
    
    mlflow.tensorflow.autolog()
    
    with mlflow.start_run():
        if model is None:

            resnet = ResNet50V2(include_top=False, weights='imagenet')

            model = Sequential()
            model.add(resnet)
            model.add(layers.GlobalAveragePooling2D())
            model.add(layers.Dense(4096, activation='relu'))
            model.add(layers.Dense(120, activation='softmax'))

        train_datagen = ImageDataGenerator(samplewise_center=True, 
                                         samplewise_std_normalization=True,
                                         rotation_range=15, zoom_range=0.2,
                                         horizontal_flip=True,
                                         validation_split=0.2
                                        )

        train_generator = train_datagen.flow_from_directory(train_dir,
                target_size=(150, 150),
                batch_size=32,
                subset='training')

        validation_generator = train_datagen.flow_from_directory(train_dir,
                target_size=(150, 150),
                batch_size=32,
                subset='validation')

        model.build()
        optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
        model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

        for layer in resnet.layers[:-10]:
            layer.trainable = True

        model.fit(train_generator, steps_per_epoch=250, epochs=40,
                validation_data=validation_generator, validation_steps=validation_generator.samples // 32)
        
        pyfunc_model = pyfunc.load_model(mlflow.get_artifact_uri("model"))

        return pyfunc_model
    

In [20]:
model = run_training()

Found 8221 images belonging to 120 classes.
Found 2001 images belonging to 120 classes.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 250 steps, validate for 62 steps
Epoch 1/40
250/250 [==============================] - 73s 293ms/step - loss: 3.1425 - accuracy: 0.2633 - val_loss: 2.3434 - val_accuracy: 0.3911
Epoch 2/40
250/250 [==============================] - 74s 295ms/step - loss: 1.6629 - accuracy: 0.5478 - val_loss: 2.1209 - val_accuracy: 0.4360
Epoch 3/40
250/250 [==============================] - 69s 276ms/step - loss: 1.1602 - accuracy: 0.6631 - val_loss: 2.0864 - val_accuracy: 0.4743
Epoch 4/40
250/250 [==============================] - 69s 275ms/step - loss: 0.7964 - accuracy: 0.7677 - val_loss: 2.1975 - val_accuracy: 0.4642
Epoch 5/40
250/250 [==============================] - 68s 273ms/step - loss: 0.5903 - accuracy: 0.8159 - val_loss: 2.2436 - val_accuracy: 0.4965
Epoch 6/40
250/250 [==============================] - 69s 275ms/step - loss: 0.4245 - accuracy